In [4]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Aug 14 10:59:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 catkin_ws
'Colab Notebooks'
'kws-fpod-yaq - 23 Jul 2022 (1).gjam'
'kws-fpod-yaq - 23 Jul 2022.gjam'
 models
'My Drive'
'sqd-ndqq-txh - 11 Agu 2022.gjam'
 Storage
'Untitled Diagram-database-GUI.drawio.png'
'Untitled Diagram.drawio'
'Untitled Diagram-Raspy-database.drawio.png'
 yolov3


In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15478, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 15478 (delta 9), reused 14 (delta 8), pack-reused 15460
Receiving objects: 100% (15478/15478), 14.08 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (10396/10396), done.


In [6]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [7]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [8]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [9]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [10]:
!echo -e "infected\nrbc\nother" > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [11]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [12]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/WIN101.JPG     
  inflating: data/obj/WIN101.txt     
  inflating: data/obj/WIN102.JPG     
  inflating: data/obj/WIN102.txt     
  inflating: data/obj/WIN103.JPG     
  inflating: data/obj/WIN103.txt     
  inflating: data/obj/WIN104.JPG     
  inflating: data/obj/WIN104.txt     
  inflating: data/obj/WIN109.JPG     
  inflating: data/obj/WIN109.txt     


In [ ]:
import glob
import os
import re
   
txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            # print(line)
            numbers = re.findall("[0-9.]+", line)
            # print(numbers)

            if numbers:
              if numbers[0] == '15':
                numbers[0] = 0
              if numbers[0] == '16':
                numbers[0] = 1
              if numbers[0] == '17':
                numbers[0] = 2
              
              # Define coordinates
              text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
              text_converted.append(text)
              # print(i, file_path)
              print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.676562 0.862963 0.072917 0.127778
0 0.768490 0.068519 0.158854 0.135185
0 0.208333 0.540278 0.070833 0.110185
0 0.494792 0.622222 0.054167 0.101852
0 0.128125 0.410648 0.041667 0.086111
0 0.044010 0.592593 0.044271 0.046296
0 0.664583 0.748148 0.058333 0.062963
0 0.798177 0.704630 0.033854 0.092593
0 0.535156 0.338889 0.039062 0.057407
0 0.362240 0.501852 0.060937 0.125926
0 0.311979 0.112963 0.035417 0.066667
0 0.157812 0.274074 0.031250 0.077778
0 0.070833 0.375463 0.035417 0.075000
0 0.304427 0.741667 0.044271 0.051852
0 0.326042 0.812500 0.053125 0.073148
0 0.051562 0.805556 0.048958 0.059259
0 0.449219 0.146759 0.036979 0.065741
0 0.505729 0.230556 0.045833 0.048148
0 0.217188 0.429167 0.045833 0.075000
0 0.157812 0.474537 0.034375 0.056481
2 0.397135 0.714352 0.103646 0.089815
2 0.963542 0.541204 0.052083 0.056481
2 0.503906 0.991204 0.035937 0.017593
0 0.208594 0.884259 0.039062 0.057407
0 0.815885 0.230093 0.034896 0.069444
1 0.876823 0.412963 0.053646 0.100000
1 0.797917 0

In [ ]:
import glob
# Don't Forget The Image Extension
images_list = glob.glob("data/obj/*.JPG")
print(images_list)
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

['data/obj/WIN109.JPG', 'data/obj/WIN104.JPG', 'data/obj/WIN101.JPG', 'data/obj/WIN102.JPG', 'data/obj/WIN103.JPG']


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-10 07:06:23--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  17.4MB/s    in 9.9s    

2022-08-10 07:06:34 (15.6 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.858470), count: 366, class_loss = 1.737117, iou_loss = 5.396819, total_loss = 7.133935 
 total_bbox = 13709381, rewritten_bbox = 0.158993 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.872584), count: 2, class_loss = 0.003366, iou_loss = 0.019663, total_loss = 0.023028 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.852388), count: 506, class_loss = 2.363269, iou_loss = 6.874229, total_loss = 9.237498 
 total_bbox = 13709889, rewritten_bbox = 0.158995 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.00000